In [2]:
import numpy as np
import matplotlib
import pandas as pd

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

train = dataset.iloc[:743,:]
test = dataset.iloc[743:,:]
print(train)
print(test)
X_train = pd.DataFrame(train.iloc[:,4:15].values)
Y_train = train.iloc[:,16:].values
print(X_train.shape)
print(Y_train.shape)
X_test = pd.DataFrame(test.iloc[:,4:15].values)
Y_test = test.iloc[:,16:].values
print(X_test.shape)
print(Y_test.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


          Date  Site  Latitude (decimal deg)  Longitude (decimal deg)  \
0    5/29/2013   WE6                 41.7134                 -83.3804   
1    5/29/2013   WE8                 41.8343                 -83.3633   
2    5/29/2013   WE4                 41.8267                 -83.1930   
3    6/10/2013   WE6                 41.7144                 -83.3805   
4    6/10/2013   WE8                 41.8353                 -83.3631   
..         ...   ...                     ...                      ...   
738  10/7/2019  WE16                 41.6599                 -83.1467   
739  10/7/2019  WE13                 41.7428                 -83.1354   
740  10/7/2019  WE12                 41.7023                 -83.2639   
741  10/7/2019   WE4                 41.8270                 -83.1949   
742  6/16/2020   WE6                 41.7054                 -83.3864   

     Secchi Depth (m)  CTD Temperature (°C)  \
0                1.60                  18.5   
1                1.00        

In [3]:
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor

2024-07-02 13:56:31.608417: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import lightgbm as lgb
def create_lightgbm():
    from sklearn.metrics import mean_squared_error as mse 
    from lightgbm import LGBMRegressor 
    
    # Create an instance of the LightGBM Regressor with the RMSE metric. 
    model = LGBMRegressor() 
    
    # Train the model using the training data. 
    # model.fit(X_train, Y_train) 
# 
    # res = model.predict(X_test) 
    return model


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [5]:
from sklearn.neural_network import MLPRegressor
def create_mlp():
    model = MLPRegressor(max_iter=1000)
    return model


In [6]:
from sklearn.linear_model import Ridge
def create_ridge():
    model = Ridge(alpha=100)
    # model.fit(X_train,Y_train)
    return model

In [7]:
from sklearn.ensemble import VotingRegressor

clf1 = create_mlp()
clf2 = create_ridge()
clf3 = create_lightgbm()

In [8]:
model = VotingRegressor(estimators=[('SVM', clf1), ('ridge', clf2), ('LightGBM', clf3)], n_jobs=5)
model.fit(X_train,Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/ensemble/_voting.py:694: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000207 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2204
[LightGBM] [Info] Number of data points in the train set: 743, number of used features: 11
[LightGBM] [Info] Start training from score 31.734267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingRegressor(estimators=[('SVM', MLPRegressor(max_iter=1000)),
                            ('ridge', Ridge(alpha=100)),
                            ('LightGBM', LGBMRegressor())],
                n_jobs=5)

In [9]:
res = model.predict(X_test)
print(res)

from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)


[25.58322728 33.67658386 21.79463848 26.04170268 20.05563617 32.03527513
 20.38483614 21.97521366 19.57715241 35.49656485 10.80248432 40.0972636
 19.09313505 28.24556665 20.27311225 46.47894495 16.7782103  43.81647097
 24.23319427 36.24187228 19.60794434 49.46137316 14.22101117 19.37941706
 21.30540467 22.04503145 51.49731027 28.07828588 52.86313127 22.39364993
 26.40062449 38.74463119 14.04686612 47.25994184 30.2212793  47.48259623
 22.6052022  20.80197685 35.34156648 13.55319044 42.56856572 31.60483662
 57.94636178 32.07943087 27.86595738 29.28062723 17.43564494 42.45517835
 35.815905   65.05034027 25.96682782 20.13554976 21.52233529 18.55562036
 33.6648099  35.2061018  45.22112387 24.29991379 16.81209953 30.01106722
 16.40008022 41.4889465  22.61543028 39.87588324 25.09552294 18.53001487
 27.19861599 12.56701752 34.91146188 29.78852136 31.91678884 25.5476311
 19.15904153 31.62690172  9.51529385 38.36332403 30.54525414 40.78520029
 21.94133928 24.59781571 29.32263124 13.38707217 18.9